In [1]:
import os
import sys
os.chdir('..')
sys.path.append(os.getcwd())

In [2]:
import torch
from Models.pretrain import *
from Models.cPB import cPB
import torch.nn.functional as F
import numpy as np
import pandas as pd
import pickle
import argparse
from torch.autograd import Variable
import pprint
import copy
import warnings
from scipy.stats import shapiro,ttest_ind,wilcoxon
warnings.filterwarnings("ignore", category=FutureWarning, module='torch.storage')
warnings.filterwarnings("ignore", category=UserWarning, module='torch.storage')
warnings.filterwarnings("ignore", category=DeprecationWarning, module='torch.storage')

In [3]:
def truncate(number, digits) -> float:
    stepper = 10.0 ** digits
    return np.trunc(stepper * number) / stepper

def print_stats(data_list, label):
    mean_val = np.mean(data_list)
    std_val = np.std(data_list)

    # Truncate values instead of rounding
    truncated_mean = truncate(mean_val, 2)
    truncated_std = truncate(std_val, 2)
    print(f"{label} Mean: {truncated_mean}%")
    print(f"{label} Std:  {truncated_std}%")
    print(f"{label} complete vale:  {truncated_mean} ± {truncated_std}")
    print(10 * '*')
    return truncated_mean, truncated_std
    
def replication(data1):
  kappa = data1

  # Apply the filtering
  before_50 = kappa[:50]  # Elements up to index 50 (exclusive)
  after_50 = kappa[50:]
  after_50_filtered = after_50[::2]
  filtered_kappa = np.concatenate((before_50, after_50_filtered))
  return filtered_kappa

# Anytime scenario



In [ ]:
base_model = 'GRU'
hidden_size = 50
dataset = 'SINE'
dataset_name = "sine_rw10_mode5_extended_16-16_1234"
mask_option = 'SUM'
model_type = 'cPB'
pretrain_task = '2'
batch_50_1 = []
batch_50_2 = []
batch_50_3 = []
batch_50_4 = []
end_1 = []
end_2 = []
end_3 = []
end_4 = []

all_results_1 = [[] for i in range(0,8)]

for i in range (1,11):
  file_1  = f'Performance/Results/{model_type}/{dataset}/Anytime2/pretrain-T{pretrain_task}_{base_model}-{dataset_name}-hidden{hidden_size}-epoch10-Mask_{mask_option}_iter{i}.pkl'
  with open(file_1, 'rb') as f:
    data1 = pickle.load(f)

  all_results_1[0].append(data1['task_1']['kappa'][50*128-1])
  all_results_1[1].append(data1['task_1']['kappa'][-1])
  all_results_1[2].append(data1['task_2']['kappa'][50*128-1])
  all_results_1[3].append(data1['task_2']['kappa'][-1])
  all_results_1[4].append(data1['task_3']['kappa'][50*128-1])
  all_results_1[5].append(data1['task_3']['kappa'][-1])
  all_results_1[6].append(data1['task_4']['kappa'][50*128-1])
  all_results_1[7].append(data1['task_4']['kappa'][-1])

  batch_50_1.append(data1['task_1']['kappa'][50*128-1])
  batch_50_2.append(data1['task_2']['kappa'][50*128-1])
  batch_50_3.append(data1['task_3']['kappa'][50*128-1])
  batch_50_4.append(data1['task_4']['kappa'][50*128-1])
  end_1.append(data1['task_1']['kappa'][-1])
  end_2.append(data1['task_2']['kappa'][-1])
  end_3.append(data1['task_3']['kappa'][-1])
  end_4.append(data1['task_4']['kappa'][-1])


Mean_list = []
var_list = []
Mean,Var = print_stats(batch_50_1, "Batch 50-1")
Mean_list.append(Mean)
var_list.append(Var)
Mean,Var = print_stats(end_1, "End 1")
Mean_list.append(Mean)
var_list.append(Var)
Mean,Var = print_stats(batch_50_2, "Batch 50-2")
Mean_list.append(Mean)
var_list.append(Var)
Mean,Var = print_stats(end_2, "End 2")
Mean_list.append(Mean)
var_list.append(Var)
Mean,Var = print_stats(batch_50_3, "Batch 50-3")
Mean_list.append(Mean)
var_list.append(Var)
Mean,Var = print_stats(end_3, "End 3")
Mean_list.append(Mean)
var_list.append(Var)
Mean,Var = print_stats(batch_50_4, "Batch 50-4")
Mean_list.append(Mean)
var_list.append(Var)
Mean,Var = print_stats(end_4, "End 4")
Mean_list.append(Mean)
var_list.append(Var)

print(f"&{Mean_list[0]} ± {var_list[0]}& {Mean_list[1]} ± {var_list[1]}& {Mean_list[2]} ± {var_list[2]}& {Mean_list[3]} ± {var_list[3]}& {Mean_list[4]} ± {var_list[4]}& {Mean_list[5]} ± {var_list[5]}& {Mean_list[6]} ± {var_list[6]}& {Mean_list[7]} ± {var_list[7]}")


In [ ]:
base_model = 'GRU'
hidden_size = 50
dataset = 'SINE'
dataset_name = "sine_rw10_mode5_extended_16-16_2341"
model_type = 'cGRU'
pretrain_task = '2'

batch_50_1 = []
batch_50_2 = []
batch_50_3 = []
batch_50_4 = []
end_1 = []
end_2 = []
end_3 = []
end_4 = []

all_results_2 = [[] for i in range(0,8)]

for i in range (1,11):
  file_1  = f'Performance/Results/{model_type}/Anytime/{dataset}/pretrain_T{pretrain_task}_{base_model}-{dataset_name}-hidden{hidden_size}-epoch10-iter{i}_periodic.pkl'
  with open(file_1, 'rb') as f:
    data1 = pickle.load(f)

  all_results_2[0].append(data1['task_1']['kappa'][50*128-1])
  all_results_2[1].append(data1['task_1']['kappa'][-1])
  all_results_2[2].append(data1['task_2']['kappa'][50*128-1])
  all_results_2[3].append(data1['task_2']['kappa'][-1])
  all_results_2[4].append(data1['task_3']['kappa'][50*128-1])
  all_results_2[5].append(data1['task_3']['kappa'][-1])
  all_results_2[6].append(data1['task_4']['kappa'][50*128-1])
  all_results_2[7].append(data1['task_4']['kappa'][-1])

  batch_50_1.append(data1['task_1']['kappa'][50*128-1])
  batch_50_2.append(data1['task_2']['kappa'][50*128-1])
  batch_50_3.append(data1['task_3']['kappa'][50*128-1])
  batch_50_4.append(data1['task_4']['kappa'][50*128-1])
  end_1.append(data1['task_1']['kappa'][-1])
  end_2.append(data1['task_2']['kappa'][-1])
  end_3.append(data1['task_3']['kappa'][-1])
  end_4.append(data1['task_4']['kappa'][-1])


Mean_list = []
var_list = []
Mean,Var = print_stats(batch_50_1, "Batch 50-1")
Mean_list.append(Mean)
var_list.append(Var)
Mean,Var = print_stats(end_1, "End 1")
Mean_list.append(Mean)
var_list.append(Var)
Mean,Var = print_stats(batch_50_2, "Batch 50-2")
Mean_list.append(Mean)
var_list.append(Var)
Mean,Var = print_stats(end_2, "End 2")
Mean_list.append(Mean)
var_list.append(Var)
Mean,Var = print_stats(batch_50_3, "Batch 50-3")
Mean_list.append(Mean)
var_list.append(Var)
Mean,Var = print_stats(end_3, "End 3")
Mean_list.append(Mean)
var_list.append(Var)
Mean,Var = print_stats(batch_50_4, "Batch 50-4")
Mean_list.append(Mean)
var_list.append(Var)
Mean,Var = print_stats(end_4, "End 4")
Mean_list.append(Mean)
var_list.append(Var)

print(f"&{Mean_list[0]} ± {var_list[0]}& {Mean_list[1]} ± {var_list[1]}& {Mean_list[2]} ± {var_list[2]}& {Mean_list[3]} ± {var_list[3]}& {Mean_list[4]} ± {var_list[4]}& {Mean_list[5]} ± {var_list[5]}& {Mean_list[6]} ± {var_list[6]}& {Mean_list[7]} ± {var_list[7]}")


In [ ]:
for i in range(0,8):
  A = all_results_1[i]
  B = all_results_2[i]
  stat_A, p_value_A = shapiro(A)
  stat_B, p_value_B = shapiro(B)
  if p_value_A >= 0.05 and p_value_B >= 0.05:
    print('Both are normal distribution')
    t_stat, p_ttest = ttest_ind(A, B, equal_var=False, alternative='greater')
    if p_ttest < 0.05:
      print('*****The mean of A is significantly greater Than B')
    else:
      print('*****The mean of B is significantly greater Than A')

  else:
    if p_value_A < 0.05 and p_value_B < 0.05:
      print('neither A or B is normally distributed')
    elif p_value_A < 0.05:
      print('B is normally distributed')
    elif p_value_B < 0.05:
      print('A is normally distributed')
    stat_wilcox, p_wilcox = wilcoxon(A, B, alternative='greater')
    if p_wilcox < 0.05:
      print('*****The mean of A is significantly greater Than B')
    else:
      print('*****The mean of B is significantly greater Than A')


In [ ]:
base_model = 'GRU'
hidden_size = 50
dataset = 'Weather'
dataset_name = "weather_st124_1conf"
mask_option = 'DOT'
model_type = 'cPB'
pretrain_task = '2'
batch_50_1 = []
batch_50_2 = []
batch_50_3 = []
batch_50_4 = []
end_1 = []
end_2 = []
end_3 = []
end_4 = []

all_results_2 = [[] for i in range(0,8)]

for i in range (1,11):
  file_1  = f'Performance/Results/{model_type}/{dataset}/Anytime2/pretrain-T{pretrain_task}_{base_model}-{dataset_name}-hidden{hidden_size}-epoch10-Mask_{mask_option}_iter{i}.pkl'
  with open(file_1, 'rb') as f:
    data1 = pickle.load(f)

  all_results_2[0].append(data1['task_1']['kappa'][50*128-1])
  all_results_2[1].append(data1['task_1']['kappa'][-1])
  all_results_2[2].append(data1['task_2']['kappa'][50*128-1])
  all_results_2[3].append(data1['task_2']['kappa'][-1])
  all_results_2[4].append(data1['task_3']['kappa'][50*128-1])
  all_results_2[5].append(data1['task_3']['kappa'][-1])
  all_results_2[6].append(data1['task_4']['kappa'][50*128-1])
  all_results_2[7].append(data1['task_4']['kappa'][-1])

  batch_50_1.append(data1['task_1']['kappa'][50*128-1])
  batch_50_2.append(data1['task_2']['kappa'][50*128-1])
  batch_50_3.append(data1['task_3']['kappa'][50*128-1])
  batch_50_4.append(data1['task_4']['kappa'][50*128-1])
  end_1.append(data1['task_1']['kappa'][-1])
  end_2.append(data1['task_2']['kappa'][-1])
  end_3.append(data1['task_3']['kappa'][-1])
  end_4.append(data1['task_4']['kappa'][-1])


Mean_list = []
var_list = []
Mean,Var = print_stats(batch_50_1, "Batch 50-1")
Mean_list.append(Mean)
var_list.append(Var)
Mean,Var = print_stats(end_1, "End 1")
Mean_list.append(Mean)
var_list.append(Var)
Mean,Var = print_stats(batch_50_2, "Batch 50-2")
Mean_list.append(Mean)
var_list.append(Var)
Mean,Var = print_stats(end_2, "End 2")
Mean_list.append(Mean)
var_list.append(Var)
Mean,Var = print_stats(batch_50_3, "Batch 50-3")
Mean_list.append(Mean)
var_list.append(Var)
Mean,Var = print_stats(end_3, "End 3")
Mean_list.append(Mean)
var_list.append(Var)
Mean,Var = print_stats(batch_50_4, "Batch 50-4")
Mean_list.append(Mean)
var_list.append(Var)
Mean,Var = print_stats(end_4, "End 4")
Mean_list.append(Mean)
var_list.append(Var)

print(f"&{Mean_list[0]} ± {var_list[0]}& {Mean_list[1]} ± {var_list[1]}& {Mean_list[2]} ± {var_list[2]}& {Mean_list[3]} ± {var_list[3]}& {Mean_list[4]} ± {var_list[4]}& {Mean_list[5]} ± {var_list[5]}& {Mean_list[6]} ± {var_list[6]}& {Mean_list[7]} ± {var_list[7]}")


In [ ]:
base_model = 'GRU'
hidden_size = 50
dataset = 'Weather'
dataset_name = "weather_st124_4conf"
model_type = 'cGRU'
pretrain_task = '1'

batch_50_1 = []
batch_50_2 = []
batch_50_3 = []
batch_50_4 = []
end_1 = []
end_2 = []
end_3 = []
end_4 = []

all_results_2 = [[] for i in range(0,8)]

for i in range (1,11):
  file_1  = f'Performance/Results/{model_type}/Anytime/{dataset}/pretrain_T{pretrain_task}_{base_model}-{dataset_name}-hidden{hidden_size}-epoch10-iter{i}_periodic.pkl'
  with open(file_1, 'rb') as f:
    data1 = pickle.load(f)

  all_results_2[0].append(data1['task_1']['kappa'][50*128-1])
  all_results_2[1].append(data1['task_1']['kappa'][-1])
  all_results_2[2].append(data1['task_2']['kappa'][50*128-1])
  all_results_2[3].append(data1['task_2']['kappa'][-1])
  all_results_2[4].append(data1['task_3']['kappa'][50*128-1])
  all_results_2[5].append(data1['task_3']['kappa'][-1])
  all_results_2[6].append(data1['task_4']['kappa'][50*128-1])
  all_results_2[7].append(data1['task_4']['kappa'][-1])

  batch_50_1.append(data1['task_1']['kappa'][50*128-1])
  batch_50_2.append(data1['task_2']['kappa'][50*128-1])
  batch_50_3.append(data1['task_3']['kappa'][50*128-1])
  batch_50_4.append(data1['task_4']['kappa'][50*128-1])
  end_1.append(data1['task_1']['kappa'][-1])
  end_2.append(data1['task_2']['kappa'][-1])
  end_3.append(data1['task_3']['kappa'][-1])
  end_4.append(data1['task_4']['kappa'][-1])


Mean_list = []
var_list = []
Mean,Var = print_stats(batch_50_1, "Batch 50-1")
Mean_list.append(Mean)
var_list.append(Var)
Mean,Var = print_stats(end_1, "End 1")
Mean_list.append(Mean)
var_list.append(Var)
Mean,Var = print_stats(batch_50_2, "Batch 50-2")
Mean_list.append(Mean)
var_list.append(Var)
Mean,Var = print_stats(end_2, "End 2")
Mean_list.append(Mean)
var_list.append(Var)
Mean,Var = print_stats(batch_50_3, "Batch 50-3")
Mean_list.append(Mean)
var_list.append(Var)
Mean,Var = print_stats(end_3, "End 3")
Mean_list.append(Mean)
var_list.append(Var)
Mean,Var = print_stats(batch_50_4, "Batch 50-4")
Mean_list.append(Mean)
var_list.append(Var)
Mean,Var = print_stats(end_4, "End 4")
Mean_list.append(Mean)
var_list.append(Var)

print(f"&{Mean_list[0]} ± {var_list[0]}& {Mean_list[1]} ± {var_list[1]}& {Mean_list[2]} ± {var_list[2]}& {Mean_list[3]} ± {var_list[3]}& {Mean_list[4]} ± {var_list[4]}& {Mean_list[5]} ± {var_list[5]}& {Mean_list[6]} ± {var_list[6]}& {Mean_list[7]} ± {var_list[7]}")


## Periodic:


In [ ]:
# Define parameters
base_model = 'GRU'
hidden_size = 250
dataset = 'SINE'
dataset_name = "sine_rw10_mode5_extended_16-16_2341"
mask_option = 'SUM'  # Fixed mask_option
model_type = 'cPB'
pretrain_task = '4'

all_results_1 = [[] for i in range(0,8)]
for i in range (1,11):
  file_1  = f'Performance/Results/{model_type}/{dataset}/Periodic/pretrain-T{pretrain_task}_{base_model}-{dataset_name}-hidden{hidden_size}-epoch10-Mask_{mask_option}_iter{i}.pkl'

  with open(file_1, 'rb') as f:
    data1 = pickle.load(f)

  all_results_1[0].append(np.mean(data1['task_1']['kappa'][0:50]))
  all_results_1[1].append(np.mean(data1['task_1']['kappa'][:]))
  all_results_1[2].append(np.mean(data1['task_2']['kappa'][0:50]))
  all_results_1[3].append(np.mean(data1['task_2']['kappa'][:]))
  all_results_1[4].append(np.mean(data1['task_3']['kappa'][0:50]))
  all_results_1[5].append(np.mean(data1['task_3']['kappa'][:]))
  all_results_1[6].append(np.mean(data1['task_4']['kappa'][0:50]))
  all_results_1[7].append(np.mean(data1['task_4']['kappa'][:]))

Mean_list = []
var_list = []
Mean,Var = print_stats(all_results_1[0], "Batch 50-1")
Mean_list.append(Mean)
var_list.append(Var)
Mean,Var = print_stats(all_results_1[1], "End 1")
Mean_list.append(Mean)
var_list.append(Var)
Mean,Var = print_stats(all_results_1[2], "Batch 50-2")
Mean_list.append(Mean)
var_list.append(Var)
Mean,Var = print_stats(all_results_1[3], "End 2")
Mean_list.append(Mean)
var_list.append(Var)
Mean,Var = print_stats(all_results_1[4], "Batch 50-3")
Mean_list.append(Mean)
var_list.append(Var)
Mean,Var = print_stats(all_results_1[5], "End 3")
Mean_list.append(Mean)
var_list.append(Var)
Mean,Var = print_stats(all_results_1[6], "Batch 50-4")
Mean_list.append(Mean)
var_list.append(Var)
Mean,Var = print_stats(all_results_1[7], "End 4")
Mean_list.append(Mean)
var_list.append(Var)

print(f"&{Mean_list[0]} ± {var_list[0]}& {Mean_list[1]} ± {var_list[1]}& {Mean_list[2]} ± {var_list[2]}& {Mean_list[3]} ± {var_list[3]}& {Mean_list[4]} ± {var_list[4]}& {Mean_list[5]} ± {var_list[5]}& {Mean_list[6]} ± {var_list[6]}& {Mean_list[7]} ± {var_list[7]}\T\B ")


print(np.round(np.mean(all_results_1[0]),3),',',np.round(np.mean(all_results_1[1]),3),',',np.round(np.mean(all_results_1[2]),3),',',np.round(np.mean(all_results_1[3]),3),',',np.round(np.mean(all_results_1[4]),3),',',np.round(np.mean(all_results_1[5]),3),',',np.round(np.mean(all_results_1[6]),3),',',np.round(np.mean(all_results_1[7]),3))


In [ ]:
# Define parameters
base_model = 'GRU'
hidden_size = 250
dataset = 'SINE'
dataset_name = "sine_rw10_mode5_extended_16-16_2341"
model_type = 'cGRU'
pretrain_task = '1'


batch_50_1_1 = []
batch_50_2_1 = []
batch_50_3_1 = []
batch_50_4_1 = []
end_1_1 = []
end_2_1 = []
end_3_1 = []
end_4_1 = []

all_results_2 = [[] for i in range(0,8)]
for i in range (1,11):
  file_1  = f'Performance/Results/{model_type}/Periodic/{dataset}/pretrain_T{pretrain_task}_{base_model}-{dataset_name}-hidden{hidden_size}-epoch10-iter{i}_periodic.pkl'

  with open(file_1, 'rb') as f:
    data1 = pickle.load(f)

  all_results_2[0].append(np.mean(data1['task_1']['kappa'][0:50]))
  all_results_2[1].append(np.mean(data1['task_1']['kappa'][:]))
  all_results_2[2].append(np.mean(data1['task_2']['kappa'][0:50]))
  all_results_2[3].append(np.mean(data1['task_2']['kappa'][:]))
  all_results_2[4].append(np.mean(data1['task_3']['kappa'][0:50]))
  all_results_2[5].append(np.mean(data1['task_3']['kappa'][:]))
  all_results_2[6].append(np.mean(data1['task_4']['kappa'][0:50]))
  all_results_2[7].append(np.mean(data1['task_4']['kappa'][:]))

Mean_list = []
var_list = []
Mean,Var = print_stats(all_results_2[0], "Batch 50-1")
Mean_list.append(Mean)
var_list.append(Var)
Mean,Var = print_stats(all_results_2[1], "End 1")
Mean_list.append(Mean)
var_list.append(Var)
Mean,Var = print_stats(all_results_2[2], "Batch 50-2")
Mean_list.append(Mean)
var_list.append(Var)
Mean,Var = print_stats(all_results_2[3], "End 2")
Mean_list.append(Mean)
var_list.append(Var)
Mean,Var = print_stats(all_results_2[4], "Batch 50-3")
Mean_list.append(Mean)
var_list.append(Var)
Mean,Var = print_stats(all_results_2[5], "End 3")
Mean_list.append(Mean)
var_list.append(Var)
Mean,Var = print_stats(all_results_2[6], "Batch 50-4")
Mean_list.append(Mean)
var_list.append(Var)
Mean,Var = print_stats(all_results_2[7], "End 4")
Mean_list.append(Mean)
var_list.append(Var)

print(f"&{Mean_list[0]} ± {var_list[0]}& {Mean_list[1]} ± {var_list[1]}& {Mean_list[2]} ± {var_list[2]}& {Mean_list[3]} ± {var_list[3]}& {Mean_list[4]} ± {var_list[4]}& {Mean_list[5]} ± {var_list[5]}& {Mean_list[6]} ± {var_list[6]}& {Mean_list[7]} ± {var_list[7]}\T\B ")


print(np.round(np.mean(all_results_2[0]),3),',',np.round(np.mean(all_results_2[1]),3),',',np.round(np.mean(all_results_2[2]),3),',',np.round(np.mean(all_results_2[3]),3),',',np.round(np.mean(all_results_2[4]),3),',',np.round(np.mean(all_results_2[5]),3),',',np.round(np.mean(all_results_2[6]),3),',',np.round(np.mean(all_results_2[7]),3))



In [ ]:
for i in range(0,8):
  A = all_results_1[i]
  B = all_results_2[i]
  stat_A, p_value_A = shapiro(A)
  stat_B, p_value_B = shapiro(B)
  if p_value_A >= 0.05 and p_value_B >= 0.05:
    print('Both are normal distribution')
    t_stat, p_ttest = ttest_ind(A, B, equal_var=False, alternative='greater')
    if p_ttest < 0.05:
      print('*****The mean of A is significantly greater Than B')
    else:
      print('*****The mean of B is significantly greater Than A')

  else:
    if p_value_A < 0.05 and p_value_B < 0.05:
      print('neither A or B is normally distributed')
    elif p_value_A < 0.05:
      print('B is normally distributed')
    elif p_value_B < 0.05:
      print('A is normally distributed')
    stat_wilcox, p_wilcox = wilcoxon(A, B, alternative='greater')
    if p_wilcox < 0.05:
      print('*****The mean of A is significantly greater Than B')
    else:
      print('*****The mean of B is significantly greater Than A')


In [ ]:
# Define parameters
base_model = 'GRU'
hidden_size = 250
dataset = 'Weather'
dataset_name = "weather_st124_3conf"
model_type = 'cGRU'
pretrain_task = '1'


batch_50_1_1 = []
batch_50_2_1 = []
batch_50_3_1 = []
batch_50_4_1 = []
end_1_1 = []
end_2_1 = []
end_3_1 = []
end_4_1 = []

all_results_2 = [[] for i in range(0,8)]
for i in range (1,11):
  file_1  = f'Performance/Results/{model_type}/Periodic/{dataset}/pretrain_T{pretrain_task}_{base_model}-{dataset_name}-hidden{hidden_size}-epoch10-iter{i}_periodic.pkl'

  with open(file_1, 'rb') as f:
    data1 = pickle.load(f)

  all_results_2[0].append(np.mean(data1['task_1']['kappa'][0:50]))
  all_results_2[1].append(np.mean(data1['task_1']['kappa'][:]))
  all_results_2[2].append(np.mean(data1['task_2']['kappa'][0:50]))
  all_results_2[3].append(np.mean(data1['task_2']['kappa'][:]))
  all_results_2[4].append(np.mean(data1['task_3']['kappa'][0:50]))
  all_results_2[5].append(np.mean(data1['task_3']['kappa'][:]))
  all_results_2[6].append(np.mean(data1['task_4']['kappa'][0:50]))
  all_results_2[7].append(np.mean(data1['task_4']['kappa'][:]))

Mean_list = []
var_list = []
Mean,Var = print_stats(all_results_2[0], "Batch 50-1")
Mean_list.append(Mean)
var_list.append(Var)
Mean,Var = print_stats(all_results_2[1], "End 1")
Mean_list.append(Mean)
var_list.append(Var)
Mean,Var = print_stats(all_results_2[2], "Batch 50-2")
Mean_list.append(Mean)
var_list.append(Var)
Mean,Var = print_stats(all_results_2[3], "End 2")
Mean_list.append(Mean)
var_list.append(Var)
Mean,Var = print_stats(all_results_2[4], "Batch 50-3")
Mean_list.append(Mean)
var_list.append(Var)
Mean,Var = print_stats(all_results_2[5], "End 3")
Mean_list.append(Mean)
var_list.append(Var)
Mean,Var = print_stats(all_results_2[6], "Batch 50-4")
Mean_list.append(Mean)
var_list.append(Var)
Mean,Var = print_stats(all_results_2[7], "End 4")
Mean_list.append(Mean)
var_list.append(Var)

print(f"&{Mean_list[0]} ± {var_list[0]}& {Mean_list[1]} ± {var_list[1]}& {Mean_list[2]} ± {var_list[2]}& {Mean_list[3]} ± {var_list[3]}& {Mean_list[4]} ± {var_list[4]}& {Mean_list[5]} ± {var_list[5]}& {Mean_list[6]} ± {var_list[6]}& {Mean_list[7]} ± {var_list[7]}\T\B ")


print(np.round(np.mean(all_results_2[0]),3),',',np.round(np.mean(all_results_2[1]),3),',',np.round(np.mean(all_results_2[2]),3),',',np.round(np.mean(all_results_2[3]),3),',',np.round(np.mean(all_results_2[4]),3),',',np.round(np.mean(all_results_2[5]),3),',',np.round(np.mean(all_results_2[6]),3),',',np.round(np.mean(all_results_2[7]),3))



In [ ]:
# Define parameters
base_model = 'GRU'
hidden_size = 50
dataset = 'Weather'
dataset_name = "weather_st124_3conf"
mask_option = 'SUM'  # Fixed mask_option
model_type = 'cPB'
pretrain_task = '1'


batch_50_1_1 = []
batch_50_2_1 = []
batch_50_3_1 = []
batch_50_4_1 = []
end_1_1 = []
end_2_1 = []
end_3_1 = []
end_4_1 = []

all_results_1 = [[] for i in range(0,8)]
for i in range (1,11):
  # file_1 = f'Performance/Results/{model_type}/{dataset}/Periodic/pretrain_T{Pretrain_task}_{base_model}-{dataset_name}-hidden{hidden_size}-epoch10-Mask_{mask_option}-iter{i}.pkl'
  file_1  = f'Performance/Results/{model_type}/{dataset}/Periodic/pretrain_T{pretrain_task}_{base_model}-{dataset_name}-hidden{hidden_size}-epoch10-Mask_{mask_option}-iter{i}.pkl'

  with open(file_1, 'rb') as f:
    data1 = pickle.load(f)

  all_results_1[0].append(np.mean(data1['task_1']['kappa'][0:50]))
  all_results_1[1].append(np.mean(data1['task_1']['kappa'][:]))
  all_results_1[2].append(np.mean(data1['task_2']['kappa'][0:50]))
  all_results_1[3].append(np.mean(data1['task_2']['kappa'][:]))
  all_results_1[4].append(np.mean(data1['task_3']['kappa'][0:50]))
  all_results_1[5].append(np.mean(data1['task_3']['kappa'][:]))
  all_results_1[6].append(np.mean(data1['task_4']['kappa'][0:50]))
  all_results_1[7].append(np.mean(data1['task_4']['kappa'][:]))

Mean_list = []
var_list = []
Mean,Var = print_stats(all_results_1[0], "Batch 50-1")
Mean_list.append(Mean)
var_list.append(Var)
Mean,Var = print_stats(all_results_1[1], "End 1")
Mean_list.append(Mean)
var_list.append(Var)
Mean,Var = print_stats(all_results_1[2], "Batch 50-2")
Mean_list.append(Mean)
var_list.append(Var)
Mean,Var = print_stats(all_results_1[3], "End 2")
Mean_list.append(Mean)
var_list.append(Var)
Mean,Var = print_stats(all_results_1[4], "Batch 50-3")
Mean_list.append(Mean)
var_list.append(Var)
Mean,Var = print_stats(all_results_1[5], "End 3")
Mean_list.append(Mean)
var_list.append(Var)
Mean,Var = print_stats(all_results_1[6], "Batch 50-4")
Mean_list.append(Mean)
var_list.append(Var)
Mean,Var = print_stats(all_results_1[7], "End 4")
Mean_list.append(Mean)
var_list.append(Var)

print(f"&{Mean_list[0]} ± {var_list[0]}& {Mean_list[1]} ± {var_list[1]}& {Mean_list[2]} ± {var_list[2]}& {Mean_list[3]} ± {var_list[3]}& {Mean_list[4]} ± {var_list[4]}& {Mean_list[5]} ± {var_list[5]}& {Mean_list[6]} ± {var_list[6]}& {Mean_list[7]} ± {var_list[7]}")


print(np.round(np.mean(all_results_1[0]),3),',',np.round(np.mean(all_results_1[1]),3),',',np.round(np.mean(all_results_1[2]),3),',',np.round(np.mean(all_results_1[3]),3),',',np.round(np.mean(all_results_1[4]),3),',',np.round(np.mean(all_results_1[5]),3),',',np.round(np.mean(all_results_1[6]),3),',',np.round(np.mean(all_results_1[7]),3))



In [ ]:
for i in range(0,8):
  A = all_results_1[i]
  B = all_results_2[i]
  stat_A, p_value_A = shapiro(A)
  stat_B, p_value_B = shapiro(B)
  if p_value_A >= 0.05 and p_value_B >= 0.05:
    print('Both are normal distribution')
    t_stat, p_ttest = ttest_ind(A, B, equal_var=False, alternative='greater')
    if p_ttest < 0.05:
      print('*****The mean of A is significantly greater Than B')
    else:
      print('*****The mean of B is significantly greater Than A')

  else:
    if p_value_A < 0.05 and p_value_B < 0.05:
      print('neither A or B is normally distributed')
    elif p_value_A < 0.05:
      print('B is normally distributed')
    elif p_value_B < 0.05:
      print('A is normally distributed')
    stat_wilcox, p_wilcox = wilcoxon(A, B, alternative='greater')
    if p_wilcox < 0.05:
      print('*****The mean of A is significantly greater Than B')
    else:
      print('*****The mean of B is significantly greater Than A')
